In [1]:
import pandas as pd
import os
import numpy as np
import form

In [2]:
in_xlsx1 = r"C:\Users\langhe\switchdrive\Institution\TIMCI\1 Project details\11 DM\3 ODK\odk_forms\06 Tanzania\2-SPA-timeflow\02-TIMCI-SPA-CGEI-ref-form\02-TIMCI-SPA-CGEI-ref-form.xlsx"
in_xlsx2 = r"C:\Users\langhe\switchdrive\Institution\TIMCI\1 Project details\11 DM\3 ODK\odk_forms\05 Senegal\2-SPA-timeflow\02-TIMCI-SPA-CGEI-ref-form\02-TIMCI-SPA-CGEI-ref-form.xlsx"

In [3]:
def read_xlsform(in_xlsx):
    
    survey_df = pd.read_excel(in_xlsx, sheet_name="survey")
    choices_df = pd.read_excel(in_xlsx, sheet_name="choices")
    settings_df = pd.read_excel(in_xlsx, sheet_name="settings")
    
    return survey_df, choices_df, settings_df

In [4]:
survey_df, choices_df, settings_df = read_xlsform(in_xlsx1)

In [5]:
def create_form_from_xls(in_xlsx):
    
    out = form.Form(in_xlsx, survey = "SPA exit interview")

    return out

In [6]:
f1 = create_form_from_xls(in_xlsx1)
f2 = create_form_from_xls(in_xlsx2)

# Comparison

In [7]:
f1.getTitle()

'02 TIMCI SPA Caregiver Exit Interview Form'

In [8]:
f2.getTitle()

'02 TIMCI EPS Entretien de Sortie Accompagnant'

In [9]:
f1.getVersion()

(0.445,)

In [10]:
f2.getVersion()

(0.441,)

In [11]:
f1.getDefaultLanguage()

('Swahili (sw)',)

In [12]:
f2.getDefaultLanguage()

('Français (fr)',)

In [13]:
print(f1.compare(f2))

Versions are different: (0.445,) and (0.441,)
Form ID is identical: 02-TIMCI-SPA-cgei


In [14]:
df1 = f1.getQuestions()
df1

,type,name,group_id,group_lbl
0,today,today,0,
1,start,start,0,
2,end,end,0,
3,deviceid,deviceid,0,
4,audit,audit,0,
...,...,...,...,...
66,select_one YESNO_DECL,m3_4,t03,
67,select_one WATER_SRC_LOC,m3_6,t03,
68,select_one TIME_2_SRC_LOC,m3_7,t03,
69,select_one FLOOR_TYPES,m3_8a,t03,


In [15]:
df2 = f2.getQuestions()
df2

,type,name,group_id,group_lbl
0,today,today,0,
1,start,start,0,
2,end,end,0,
3,deviceid,deviceid,0,
4,audit,audit,0,
...,...,...,...,...
71,text,m3_5o,t03,
72,select_one WATER_SRC_LOC,m3_6,t03,
73,select_one TIME_2_SRC_LOC,m3_7,t03,
74,select_one FLOOR_TYPES,m3_8a,t03,


In [16]:
f1.addedQuestions(f2)

,type_y,name,group_id_y,group_lbl_y
71,select_one YESNO,i4_3,i4,
72,select_one TREATMENT,h4_2,h4,
73,integer,m3_2,t03,
74,select_one COOKING_FUEL,m3_5,t03,
75,text,m3_5o,t03,


In [17]:
f1.removedQuestions(f2)

,type_x,name,group_id_x,group_lbl_x


# Tests

In [18]:
choices_df["list_name"].dropna().unique()

array(['YESNO', 'YESNO_DECL', 'OXIMETRY_RATIONALE', 'LIKERT',
       'SATISFACTION_LIKERT', 'CONFIDENCE_LIKERT', 'CDSA_EXPLANATION',
       'REFERRAL_TIME', 'REFERRAL_LOCATION', 'REFERRAL_ADHERENCE',
       'FACILITY_ALTERNATIVE', 'TREATMENT_ALTERNATIVE',
       'ADVICE_ALTERNATIVE', 'TREATMENT', 'TREATMENT_EXPLANATION',
       'FWUP_INTENTION', 'TREATMENT_COSTS', 'MISSED_WORKDAYS', 'FEE_CAT',
       'FEEDING_METHODS', 'CLOSEST_FACILITY', 'RFVSF', 'HOUSEHOLD_HEAD',
       'COOKING_FUEL', 'WATER_SRC_LOC', 'TIME_2_SRC_LOC', 'FLOOR_TYPES',
       'ROOF_TYPES', 'MAIN_TOILET'], dtype=object)

In [19]:
survey_df.shape

(119, 36)

In [20]:
tmp = survey_df[survey_df["name"].notnull()]
tmp = tmp[tmp["type"] != "note"]
tmp = tmp[["type", "name", "label::English (en)"]]
tmp = tmp.assign(group_id = "",
                 group_lbl = "")
tmp

,type,name,label::English (en),group_id,group_lbl
0,today,today,NaN,,
1,start,start,NaN,,
2,end,end,NaN,,
3,deviceid,deviceid,NaN,,
4,audit,audit,NaN,,
...,...,...,...,...,...
112,select_one YESNO_DECL,m3_4,j22) Is this toilet shared with another househ...,,
113,select_one WATER_SRC_LOC,m3_6,j24) Where is the household's main source of d...,,
114,select_one TIME_2_SRC_LOC,m3_7,j25) How much time does it take to the househo...,,
115,select_one FLOOR_TYPES,m3_8a,j26) What type of floor do you have at home?,,


In [21]:
group_ids = [0]
for index, row in tmp.iterrows():
    if row["type"] == "begin_group":
        group_ids.append(row["name"]) # Append the name to the list 'g'
    elif row["type"] == "end_group":
        if len(group_ids) > 1:
            group_ids = group_ids[:-1]
    else:
        tmp.loc[index, "group_id"] = group_ids[-1]
tmp = tmp[(tmp["type"] != "begin_group") & (tmp["type"] != "end_group")]
tmp

,type,name,label::English (en),group_id,group_lbl
0,today,today,NaN,0,
1,start,start,NaN,0,
2,end,end,NaN,0,
3,deviceid,deviceid,NaN,0,
4,audit,audit,NaN,0,
...,...,...,...,...,...
112,select_one YESNO_DECL,m3_4,j22) Is this toilet shared with another househ...,t03,
113,select_one WATER_SRC_LOC,m3_6,j24) Where is the household's main source of d...,t03,
114,select_one TIME_2_SRC_LOC,m3_7,j25) How much time does it take to the househo...,t03,
115,select_one FLOOR_TYPES,m3_8a,j26) What type of floor do you have at home?,t03,


In [22]:
survey_df[4:]

,type,name,label::English (en),label::Français (fr),label::Swahili (sw),label::မြန်မာစာ (my),label::हिन्दी (in),appearance,relevant,choice_filter,...,constraint_message::Français (fr),constraint_message::Swahili (sw),constraint_message::မြန်မာစာ (my),constraint_message::हिन्दी (in),media::image::English (en),media::image::Français (fr),media::image::Swahili (sw),media::image::မြန်မာစာ (my),media::image::हिन्दी (in),trigger
4,audit,audit,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,begin_group,front_page,Front page,Page de garde,NaN,NaN,NaN,field-list,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,note,NaN,"<h3 style=""text-align: center;""><span style=""...","<h3 style=""text-align: center;""><span style=""...","<h3 style=""text-align: center;""><span style=""...","<h3 style=""text-align: center;""><span style=""...","<h3 style=""text-align: center;""><span style=""...",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,timci1.png,timci1.png,timci1.png,timci1.png,timci1.png,NaN
7,note,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,bottom.png,bottom.png,bottom.png,bottom.png,bottom.png,NaN
8,end_group,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114,select_one TIME_2_SRC_LOC,m3_7,j25) How much time does it take to the househo...,j25) Combien de temps faut-il à la famille pou...,j25) Inachukua muda gani kwa kaya kufikia chan...,အိမ်မှ သောက်သုံးရေ/ချက်ပြုတ်ခြင်း အတွက် ရေရယူရ...,NaN,NaN,${m3_6}=2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
115,select_one FLOOR_TYPES,m3_8a,j26) What type of floor do you have at home?,j26) Quel type de sol avez-vous dans votre mai...,j26) Je! una sakafu ya aina gani nyumbani?,အိမ်၏ ကြမ်းခင်းကို မည်သည့် ပစ္စည်းများဖြင့် ပြ...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
116,select_one ROOF_TYPES,m3_9a,j27) What type of roof do you have at home ?,j27) Quel type de toit de avez-vous dans votre...,j27) Una aina gani ya paa nyumbani?,အိမ်၏ ခေါင်မိုးကို မည်သည့် ပစ္စည်းများဖြင့် ပြ...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
117,note,NaN,You have reached the end of this page. When yo...,Vous avez atteint la fin de cette page. Quand ...,"Umefika mwisho wa ukurasa huu. Ukimaliza, tafa...",ဤစာမျက်နှာ၏အဆုံးသို့ရောက်ပါပြီ။ သင်ပြီးဆုံးပါက...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,bottom.png,bottom.png,bottom.png,bottom.png,bottom.png,NaN


In [23]:
test = [+]

SyntaxError: invalid syntax (Temp/ipykernel_940/3361486175.py, line 1)